# Transforming SFHAs

This notebook experiments with transforming data from REST API queries in-place, either in pandas dataframes or python dicts.

## Extracting corrected LOMRs and all flood delineations inside those LOMRs

In [ ]:
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureSet, FeatureLayer, FeatureLayerCollection, SpatialDataFrame
from arcgis.geometry import Geometry, Point, filters, union, buffer
import pandas as pd

In [ ]:
# Feature services
city_lims_url = "https://maps.bouldercolorado.gov/arcgis/rest/services/plan/CityLimits/MapServer/0"
city = FeatureLayer(city_lims_url)

nfhl_url = "https://hazards.fema.gov/gis/nfhl/rest/services/public/NFHL/MapServer"
nfhl = FeatureLayerCollection(nfhl_url)
lomr = nfhl.layers[1]
sfha = nfhl.layers[27]

In [ ]:
sr = 2876 # NAD83(HARN) / Colorado North (ftUS)

In [ ]:
# Geometry filter object
anon_gis = GIS()
city_lims = city.query(out_sr=sr)
city_geoms = [poly.geometry for poly in city_lims.features]
city_union = union(spatial_ref=sr, geometries=city_geoms, gis=anon_gis)
geom_filter = filters.intersects(city_union, sr=sr)

In [ ]:
# Query
date_str = '2018-08-16'
clause = f"STATUS = 'Effective' AND EFF_DATE >= '{date_str}'"
boulder_lomrs = lomr.query(where=clause,
                           geometry_filter=geom_filter,
                           out_sr=sr)

In [ ]:
# Drop LOMR polygons that have duplicate Case Numbers and Geometries.
temp = boulder_lomrs.sdf
temp['GEOM_STR'] = str(temp['SHAPE'])
temp.drop_duplicates(subset=['CASE_NO', 'GEOM_STR'], inplace=True)
temp.sort_values(by='EFF_DATE', inplace=True, ascending=False)
boulder_lomrs = FeatureSet.from_dataframe(temp)

In [ ]:
# Query BoCo flood areas
flood_areas = sfha.query(where="DFIRM_ID = '08013C' AND ZONE_SUBTY <> 'AREA OF MINIMAL FLOOD HAZARD'",
                         out_fields=['FLD_AR_ID', 'STUDY_TYP', 'FLD_ZONE', 'ZONE_SUBTY', 'SFHA_TF', 'STATIC_BFE', 'DEPTH'],
                         out_sr=sr)

In [ ]:
# Copy BoCo flood areas and empty the df
flood_sdf = flood_areas.sdf
subset = flood_sdf.copy()
subset.drop(list(range(len(subset))),inplace=True)

In [ ]:
# Loop through LOMRs and flood areas to find polys inside LOMRs
for l in boulder_lomrs.features:
    g = Geometry(l.geometry)
    
    # buffer LOMR geom by one foot to avoid topological
    # errors where polys share an edge
    buf = g.buffer(1)

    for row in flood_areas.features:
        area_id = row.attributes['FLD_AR_ID']
        f = Geometry(row.geometry)
        if buf.contains(f):
            subset = subset.append(flood_sdf[flood_sdf['FLD_AR_ID'] == area_id],
                                               ignore_index=True)

# drop any rows that represent duplicate flood areas
subset.drop_duplicates(subset=['FLD_AR_ID'], inplace=True)
subset = subset.set_index("FLD_AR_ID")

## Transform the SFHAs

### Criteria:
- FLOODPLAIN
  - 500-Year = "FLD_ZONE = 'X' AND ZONE_SUBTY IN ('0.2 PCT ANNUAL CHANCE FLOOD HAZARD', 'LEVEE')"
  - 100-Year = "SFHA_TF = 'T'"
    - Conveyance Zone = "SFHA_TF = 'T' AND 'ZONE_SUBTY' = 'FLOODWAY'"
- DRAINAGE
  - Use "set-theoretic" funcs: centroid inside city-floodplain with DRAINAGE = X
- LIFECYCLE = 'Active'
- ADOPTDATE = lomr enacted date. Scrape this during extract phase
- FEMAZONE
  - if FLD_ZONE = AO
    - FEMAZONE = FLD_ZONE + str(DEPTH)
  - if FLD_ZONE = AH
    - FEMAZONE = FLD_ZONE + str(STATIC_BFE)
- SOURCE = 'FEMA'
- INEFFDATE = Null


In [ ]:
# Rough spatial join of LOMRs and SFHAs
spat_join = boulder_lomrs.sdf.spatial.join(flood_sdf)[["FLD_AR_ID", "EFF_DATE"]]
# Sort by effective date, most recent first
spat_join.sort_values(by=['EFF_DATE'], inplace=True, ascending=False)
# Remove duplicate FLD_AR_IDs, keep the first dup record encountered.
spat_join.drop_duplicates(subset=['FLD_AR_ID'], inplace=True, keep='first')
spat_join.rename(columns={"EFF_DATE": "ADOPTDATE"}, inplace=True)
spat_join = spat_join.set_index('FLD_AR_ID')

In [ ]:
city_flood_url = "https://maps.bouldercolorado.gov/arcgis3/rest/services/util/Floodplain/MapServer/3"
city_flood = FeatureLayer(city_flood_url)
sfha_compare = city_flood.query(out_fields=['DRAINAGE'], out_sr=2876, as_df=True)

In [ ]:
def union_drainages(row):
    no_sr = union(geometries=row["SHAPE"], spatial_ref=sr, gis=anon_gis)
    geom = Geometry({"rings": no_sr.rings, "spatialReference": {"wkid": sr}})
    return geom

drainages = pd.DataFrame(sfha_compare.groupby("DRAINAGE")["SHAPE"].apply(list)).reset_index()
drainages["SHAPE"] = drainages.apply(union_drainages, axis=1)
drain_fs = FeatureSet.from_dataframe(drainages)

In [ ]:
def calc_floodplain(row):
    if row["FLD_ZONE"] == 'X' and row["ZONE_SUBTY"] in ('0.2 PCT ANNUAL CHANCE FLOOD HAZARD', 'AREA WITH REDUCED FLOOD RISK DUE TO LEVEE'):
        floodplain = '500-Year'
    if row["SFHA_TF"] == "T":
        if row["ZONE_SUBTY"] == 'FLOODWAY':
            floodplain = 'Conveyance Zone'
        else:
            floodplain = '100-Year'
    return floodplain

def calc_femazone(row):
    if row["FLD_ZONE"] == 'AO':
        zone = 'AO' + str(row['DEPTH'])
    elif row["FLD_ZONE"] == 'AH':
        zone = 'AH' + STR(row["STATIC_BFE"])
    else:
        zone = row["FLD_ZONE"]
    return zone

def calc_drainage(row):
    for item in drain_fs.features:
        x, y = row["SHAPE"].centroid
        centroid = Geometry({"x": x, "y": y, "spatialReference": {"wkid": sr}})
        drain = item.attributes["DRAINAGE"]
        g = Geometry(item.geometry)
        if g.contains(centroid):
            return drain


subset["FLOODPLAIN"] = subset.apply(calc_floodplain, axis=1)
subset["FEMAZONE"] = subset.apply(calc_femazone, axis=1)
subset["DRAINAGE"] = subset.apply(calc_drainage, axis=1)
subset["INEFFDATE"] = None
subset["LIFECYCLE"] = "Active"
subset["SOURCE"] = "FEMA"
subset = subset.join(spat_join) #  To calc ADOPTDATE